# Evaluation of our predictions

## Imports

In [1]:
import os
import time

import cv2
import numpy as np
import torch
from torch.utils.data.sampler import SequentialSampler
from torch_geometric.data import DataLoader

from pg_networks.gcn import GCN
import src.config as cfg
from src.davis_2016 import DAVIS2016
from src.metrics import db_eval_iou, db_eval_boundary, db_eval_t_stab
from src.vis_utils import compute_combo_img

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=3)

## Choose model

In [2]:
model_path = '15057.182069204_best_model.pth'

## Paths & Constants

## Eval Dataset

In [3]:
val = DAVIS2016(cfg.PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                cfg.CONTOURS_FOLDERS_PATH, cfg.IMAGES_AUGMENTED_FOLDERS_PATH, cfg.TRANSLATIONS_FOLDERS_PATH, 
                cfg.LAYER, cfg.K, cfg.EPOCHS_WO_AVEGRAD, 1,
                cfg.SKIP_SEQUENCES, cfg.TRAIN_SEQUENCES[:], cfg.VAL_SEQUENCES[15:],
                train=False)

In [4]:
print("Val size: %i" % len(val))

Val size: 357


## Load GCN Model

In [5]:
model = GCN(in_channels=val[0].num_features, out_channels=val[0].y.shape[1])
model.load_state_dict(torch.load(model_path))
model.eval()
model.double()

GCN(
  (conv1): GCNConv(256, 512)
  (conv2): GCNConv(512, 512)
  (conv3): GCNConv(512, 1024)
  (lin1): Linear(in_features=1024, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=2, bias=True)
)

## Evaluate Model

In [6]:
val_loader = DataLoader(val, batch_size=1, shuffle=False, sampler=SequentialSampler(val))
file_names = val.processed_file_names

for i, data in enumerate(val_loader):
    
    #if i > 0: break
    
    # Forward pass to get outputs
    with torch.no_grad():
        translation_0_1_pred = model(data)
        
    # Get folder and filename for images
    try:
        file_name_1 = file_names[i + 1]    
    # Break if file_name_1 is last frame
    except IndexError as e:
        mean_J_combo = running_J_combo / running_index
        mean_J_osvos = running_J_osvos / running_index
        mean_F_combo = running_F_combo / running_index
        mean_F_osvos = running_F_osvos / running_index
        
        print(folder_old)
        print('\tmean_J_combo: {}, mean_J_osvos: {}'.format(mean_J_combo, mean_J_osvos))
        print('\tmean_F_combo: {}, mean_F_osvos: {}'.format(mean_F_combo, mean_F_osvos))
        break
        
    folder_1 = file_name_1[:-11]
    augmentation_count_1 = file_name_1[-10:-9]
    file_name_1 = file_name_1[-8:-3]
    
    # Load OSVOS result image
    osvos_img_1_path = os.path.join(cfg.OSVOS_RESULTS_FOLDERS_PATH, folder_1,
                                    ('{}{}'.format(file_name_1, '.png')))
    osvos_img_1 = cv2.imread(osvos_img_1_path)
    osvos_img_1_gray = cv2.imread(osvos_img_1_path, cv2.IMREAD_GRAYSCALE)
    
    # Load ground truth annotation
    annotation_img_1_path = os.path.join(cfg.ANNOTATIONS_AUGMENTED_FOLDERS_PATH, folder_1, augmentation_count_1,
                                         ('{}{}'.format(file_name_1, '.png')))
    annotation_img_1 = cv2.imread(annotation_img_1_path, cv2.IMREAD_GRAYSCALE)
    
    # Get contour
    contour_0 = data.contour
    contour_1_pred = contour_0.type(torch.DoubleTensor).add(translation_0_1_pred)
    
    # Create combined image
    _, combo_img_1, _, _ = compute_combo_img(contour_1_pred, osvos_img_1)
    combo_img_1_path = os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, folder_1, 
                                    ('{}{}'.format(file_name_1, '.png')))
    
    if not os.path.exists(os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, folder_1)):
        os.makedirs(os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, folder_1))
        
    cv2.imwrite(combo_img_1_path, combo_img_1*255)
    
    #Compute J
    J_combo = db_eval_iou(annotation_img_1, combo_img_1)
    J_osvos = db_eval_iou(annotation_img_1, osvos_img_1_gray)
    
    #Compute F
    F_combo = db_eval_boundary(combo_img_1, annotation_img_1)
    F_osvos = db_eval_boundary(osvos_img_1_gray, annotation_img_1)
    
    # Compute metrics per sequence
    if i == 0:
        running_index = 0
        running_J_combo = 0.
        running_J_osvos = 0.
        running_F_combo = 0.
        running_F_osvos = 0.
        folder_old = folder_1

    # Update metrics if sequence is the same
    if folder_old == folder_1:
        running_index += 1
        running_J_combo += J_combo
        running_J_osvos += J_osvos
        running_F_combo += F_combo
        running_F_osvos += F_osvos
    # Calculate mean, print, and reset if new sequence
    else:
        mean_J_combo = running_J_combo / running_index
        mean_J_osvos = running_J_osvos / running_index
        mean_F_combo = running_F_combo / running_index
        mean_F_osvos = running_F_osvos / running_index
        
        print(folder_old)
        print('\tmean_J_combo: {}, mean_J_osvos: {}'.format(mean_J_combo, mean_J_osvos))
        print('\tmean_F_combo: {}, mean_F_osvos: {}'.format(mean_F_combo, mean_F_osvos))
        
        folder_old = folder_1
        running_index = 0
        running_J_combo = 0.
        running_J_osvos = 0.
        running_F_combo = 0.
        running_F_osvos = 0.
        

motocross-jump
	mean_J_combo: 0.6125047943802047, mean_J_osvos: 0.6503471016028604
	mean_F_combo: 0.5384077503995768, mean_F_osvos: 0.5875214975023312
paragliding-launch
	mean_J_combo: 0.6099725472024609, mean_J_osvos: 0.6174057789598415
	mean_F_combo: 0.5956510269013316, mean_F_osvos: 0.6027452645867117
parkour
	mean_J_combo: 0.806116636962922, mean_J_osvos: 0.8794923903394034
	mean_F_combo: 0.8340619633277734, mean_F_osvos: 0.8688266479644037
scooter-black
	mean_J_combo: 0.5015314384957784, mean_J_osvos: 0.5377931169400938
	mean_F_combo: 0.6366506172199692, mean_F_osvos: 0.6685280987377937
soapbox
	mean_J_combo: 0.8372865636175877, mean_J_osvos: 0.8580771285333614
	mean_F_combo: 0.8127218475511981, mean_F_osvos: 0.8259189933398496
